# Assignment 3 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg

In [1]:
#import statements
import numpy as np;
import pandas as pd;
import plotnine as pn;

#set constants
A,B,c = 2.2e-4,2.7e-6,1.124

(A,B,c)

(0.00022, 2.7e-06, 1.124)

## Part [a] - Repeated Simpson's Rule for integration in Numpy

### Density function for future life Expectancy
As stated in the problem, I assume that mortality follows the Makeham law, or:
$$f_{T_x}(t) = (A + Bc^{x + t})e^{-At -\frac{B}{\log(c)}c^x(c^t - 1)}$$
Here, $x$ is the current age of the person, $t$ is the number of years they have left to live, and $A,B,c$ are constants, as defined above. I code this function below.

In [2]:
#Equation for probability density of living t more years given current age x
Makeham = lambda t,x,A = A,B = B,c = c : \
    (A + B*(c**(x + t)))*np.exp(-A*t - (B/np.log(c))*(c**x)*((c**t)- 1))

#Create version multiplied by t for purposes of expectation
multMakeham = lambda t,x,A = A,B = B,c = c : \
      t * (A + B*(c**(x + t)))* np.exp(-A*t - (B/np.log(c))*(c**x)*((c**t)- 1))


In [3]:
#Function for simpson's Method on Makeham, given the age
def Simpsons(func, currAge, a = 0, n = 50):
    #set upper bound of integration
    b = 130 - currAge;
    h = (b - a)/(2*n)
    
    #create arrays for function input for I
    input_1 = np.array([a + (2*j - 1)*h for j in range(1, n + 1)])
    input_2 = np.array([a + 2*j*h for j in range(1, n)])

    #Simpson's Rule approximation
    I = (h/3)*(func(a, currAge) + 4 * np.sum([func(t, currAge) for t in input_1]) +\
                2 * np.sum([func(t, currAge) for t in input_2]) + func(b, currAge))
    return I;


In [4]:
#Create array of ages to evaluate
Ages = np.linspace(0, 100, 11);

#Create array for expected years left
Exp_Fut_life = np.array([Simpsons(multMakeham, age) for age in Ages]);

#Create area for expected death age
Exp_Death_date = Ages + Exp_Fut_life;

data = pd.DataFrame(np.array([Ages, Exp_Fut_life, Exp_Death_date]).T, \
                    columns = ["current age", "expected future lifetime", "expected age at death"])

data

,current age,expected future lifetime,expected age at death
0,0.0,85.564247,85.564247
1,10.0,75.745777,85.745777
2,20.0,65.913131,85.913131
3,30.0,56.079203,86.079203
4,40.0,46.277622,86.277622
5,50.0,36.591443,86.591443
6,60.0,27.209687,87.209687
7,70.0,18.510407,88.510407
8,80.0,11.103323,91.103323
9,90.0,5.652803,95.652803
